# Check that the Voronoi-binned data has been saved correctly in `make_df_sami.py`
---

In [1]:
%matplotlib widget

In [18]:
# Imports
import sys
import os 
import numpy as np
import pandas as pd
from astropy.visualization import hist
from astropy.io import fits
from tqdm import tqdm
from scipy import constants
from scipy.stats import ks_2samp, anderson_ksamp, spearmanr

from spaxelsleuth.loaddata.lzifu import load_lzifu_galaxies
from spaxelsleuth.loaddata.sami import load_sami_galaxies
from spaxelsleuth.plotting.plottools import plot_empty_BPT_diagram
from spaxelsleuth.plotting.plottools import vmin_fn, vmax_fn, label_fn, cmap_fn, fname_fn
from spaxelsleuth.plotting.plottools import bpt_colours, bpt_labels, whav_colors, whav_labels
from spaxelsleuth.plotting.plottools import morph_labels, morph_ticks
from spaxelsleuth.plotting.plottools import ncomponents_labels, ncomponents_colours
from spaxelsleuth.plotting.plottools import component_labels, component_colours
from spaxelsleuth.plotting.plotgalaxies import plot2dhistcontours, plot2dscatter, plot2dcontours
from spaxelsleuth.plotting.plot2dmap import plot2dmap
from spaxelsleuth.plotting.sdssimg import plot_sdss_image
from spaxelsleuth.plotting.plottools import plot_empty_BPT_diagram, plot_BPT_lines

import matplotlib
from matplotlib import rc, rcParams
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from IPython.core.debugger import Tracer

rc("text", usetex=False)
rc("font",**{"family": "serif", "size": 11})
rcParams["savefig.bbox"] = "tight"
rcParams["savefig.format"] = "pdf"
plt.ion()
plt.close("all")


In [7]:
# Paths
sami_data_path = os.environ["SAMI_DIR"]
assert "SAMI_DIR" in os.environ, "Environment variable SAMI_DIR is not defined!"
sami_datacube_path = os.environ["SAMI_DATACUBE_DIR"]
assert "SAMI_DATACUBE_DIR" in os.environ, "Environment variable SAMI_DATACUBE_DIR is not defined!"


In [3]:
# Options
fig_path = "/priv/meggs3/u5708159/SAMI/figs/paper/"
savefigs = True
ncomponents = "recom"   # Options: "1" or "recom"
eline_SNR_min = 5       # Minimum S/N of emission lines to accept
plt.close("all")


In [4]:
# Load the ubinned data 
df_unbinned = load_sami_galaxies(ncomponents=ncomponents,
                        bin_type="default",
                        eline_SNR_min=eline_SNR_min, 
                        vgrad_cut=False,
                        line_amplitude_SNR_cut=True,
                        correct_extinction=False,
                        sigma_gas_SNR_cut=True)


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc,

In [6]:
# Load the ubinned data 
df_binned = load_sami_galaxies(ncomponents=ncomponents,
                        bin_type="adap",
                        eline_SNR_min=eline_SNR_min, 
                        vgrad_cut=False,
                        line_amplitude_SNR_cut=True,
                        correct_extinction=False,
                        sigma_gas_SNR_cut=True)


In [13]:
###########################################################################
# Load the DataFrame containing S/N metadata
###########################################################################
# Load the SNR DataFrame.
df_snr = pd.read_csv(os.path.join(sami_data_path, "sample_summary.csv"))

# Sort by median red S/N in 2R_e
df_snr = df_snr.sort_values("Median SNR (R, 2R_e)", ascending=False)

# Set index to catid for ease of indexing
df_snr = df_snr.set_index("ID")

## TEST 1: 2D map plots for a specific galaxy. Do they look the same?
---

In [33]:
gal = 100162

In [34]:
###########################################################################
# Collage figure 1: coloured by number of components
###########################################################################
markers = ["o", ">", "D"]
l = 0.05
b = 0.05
dw = 0.05
dh = 0.1
w = (1 - 2 * l - 2 * dw) / 5
h = (1 - 2 * b - dh) / 2

###########################################################################
# Define galaxy
###########################################################################
# Check validity
assert gal in df_unbinned["ID"], f"{gal} not found in SAMI sample!"

# Load the DataFrame
df_gal = df_unbinned[df_unbinned["ID"] == gal]
df_gal.loc[df_gal["Number of components"] == 0, "Number of components"] = np.nan

###########################################################################
# Create the figure
###########################################################################
fig_collage = plt.figure(figsize=(18, 7))
ax_sdss = fig_collage.add_axes([l, b, w, h])
ax_im = fig_collage.add_axes([l, b + h + dh, w, h])
bbox = ax_im.get_position()
cax_im = fig_collage.add_axes([bbox.x0 + bbox.width * 0.035, bbox.y0 + bbox.height, bbox.width * 0.93, 0.025])
axs_bpt = []
axs_bpt.append(fig_collage.add_axes([l + w + dw, b + h + dh, w, h]))
axs_bpt.append(fig_collage.add_axes([l + w + dw + w, b + h + dh, w, h]))
axs_bpt.append(fig_collage.add_axes([l + w + dw + 2 * w, b + h + dh, w, h]))
axs_whav = []
axs_whav.append(fig_collage.add_axes([l + w + dw, b, w, h]))
axs_whav.append(fig_collage.add_axes([l + w + dw + w, b, w, h]))
axs_whav.append(fig_collage.add_axes([l + w + dw + 2 * w, b, w, h]))

###########################################################################
# Plot SDSS image and component map
###########################################################################
col_z = "Number of components"

# SDSS image
res = plot_sdss_image(df_gal, ax=ax_sdss)
if res is None:
    ax_sdss.text(s="Galaxy not in SDSS footprint",
                 x=0.5, y=0.5, horizontalalignment="center",
                 transform=ax_sdss.transAxes)

# Plot the number of components fitted.
plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
          PA_deg=0,
          col_z=col_z, 
          ax=ax_im, cax=cax_im, cax_orientation="horizontal", show_title=False)

# Text string showing basic info
sfr = df_snr.loc[gal, 'SFR (component 0)']
mstar = df_gal["log M_*"].unique()[0]
if np.isnan(sfr):
    sfr = "n/a"
else:
    sfr = f"{sfr:.3f}" + r" $\rm M_\odot\,yr^{-1}$"
    
if np.isnan(mstar):
    mstar = r"$\log \rm \, M_\odot = $ n/a"
else:
    mstar = r"$\log \rm \, M_\odot = $" + f"{mstar:.2f}"

t = axs_bpt[0].text(s=f"{gal}, {df_snr.loc[gal, 'Morphology']}, SFR = {sfr}, {mstar}, SNR = {df_snr.loc[gal, 'Median SNR (R, 2R_e)']:.2f}", 
    x=0.0, y=1.02, transform=axs_bpt[0].transAxes)

# Plot BPT diagram
col_y = "log O3"
for cc, col_x in enumerate(["log N2", "log S2", "log O1"]):
    # Plot full SAMI sample
    plot2dhistcontours(df=df_unbinned, 
                       col_x=f"{col_x} (total)",
                       col_y=f"{col_y} (total)", col_z="count", log_z=True,
                       alpha=0.5, cmap="gray_r",
                       ax=axs_bpt[cc], plot_colorbar=False)

    # Add BPT functions
    plot_BPT_lines(ax=axs_bpt[cc], col_x=col_x)    

    # Plot measurements for this galaxy
    plot2dscatter(df=df_gal,
                  col_x=f"{col_x} (total)",
                  col_y=f"{col_y} (total)",
                  col_z=None if col_z == "Number of components" else col_z,
                  marker=markers[0], ax=axs_bpt[cc], 
                  cax=None,
                  markersize=20, 
                  markerfacecolor=component_colours[0] if col_z == "Number of components" else None, 
                  markeredgecolor="black",
                  plot_colorbar=False)

# Decorations
[ax.grid() for ax in axs_bpt]
[ax.set_ylabel("") for ax in axs_bpt[1:]]
[ax.set_yticklabels([]) for ax in axs_bpt[1:]]
[ax.set_xticks(ax.get_xticks()[:-1]) for ax in axs_bpt[:-1]]
for ax in axs_bpt:
    _ = [c.set_rasterized(True) for c in ax.collections]
_ = [c.set_rasterized(True) for c in ax_im.collections]

###########################################################################
# Plot WHAN, WHAV and WHAV* diagrams.
###########################################################################
# Plot LZIFU measurements
for cc, col_x in enumerate(["log N2", "sigma_gas - sigma_*", "v_gas - v_*"]):
    # Plot full SAMI sample
    plot2dhistcontours(df=df_unbinned, 
                       col_x=f"{col_x} (total)" if col_x == "log N2" else f"{col_x}",
                       col_y=f"log HALPHA EW (total)" if col_x == "log N2" else f"log HALPHA EW",
                       col_z="count", log_z=True,
                       alpha=0.5, cmap="gray_r", ax=axs_whav[cc],
                       plot_colorbar=False)

# WHAN diagram
plot2dscatter(df=df_gal,
              col_x=f"log N2 (total)",
              col_y=f"log HALPHA EW (total)",
              col_z=None if col_z == "Number of components" else col_z,
              marker=markers[0], ax=axs_whav[0], 
              cax=None,
              markersize=20, 
              markerfacecolor=component_colours[0] if col_z == "Number of components" else None, 
              markeredgecolor="black",
              plot_colorbar=False)

# Kinematics 
for cc, col_x in enumerate(["sigma_gas - sigma_*", "v_gas - v_*"]):
    # Plot the data for this galaxy
    for ii in range(3):
        plot2dscatter(df=df_gal,
                      col_x=f"{col_x} (component {ii})",
                      col_y=f"log HALPHA EW (component {ii})",
                      col_z=None if col_z == "Number of components" else col_z,
                      marker=markers[ii], ax=axs_whav[cc + 1], 
                      cax=None,
                      markersize=20, 
                      markerfacecolor=component_colours[ii] if col_z == "Number of components" else None, 
                      markeredgecolor="black",
                      plot_colorbar=False)

# Decorations
[ax.grid() for ax in axs_whav]
[ax.set_ylabel("") for ax in axs_whav[1:]]
[ax.set_yticklabels([]) for ax in axs_whav[1:]]
[ax.set_xticks(ax.get_xticks()[:-1]) for ax in axs_whav[:-1]]
[ax.axvline(0, ls="--", color="k") for ax in axs_whav[1:]]
for ax in axs_whav:
    _ = [c.set_rasterized(True) for c in ax.collections]

# Legend
legend_elements = [Line2D([0], [0], marker=markers[ii], 
                          color="none", markeredgecolor="black",
                          label=f"Component {ii}",
                          markerfacecolor=component_colours[ii], markersize=5) for ii in range(3)]
axs_bpt[-1].legend(handles=legend_elements, fontsize="x-small", loc="upper right")


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gal["x, y (pixels)"] = list(zip(df_gal["x (projected, arcsec)"] / as_per_px, df_gal["y (projected, arcsec)"] / as_per_px))


In [35]:
###########################################################################
# Collage figure 1: coloured by number of components
###########################################################################
markers = ["o", ">", "D"]
l = 0.05
b = 0.05
dw = 0.05
dh = 0.1
w = (1 - 2 * l - 2 * dw) / 5
h = (1 - 2 * b - dh) / 2

###########################################################################
# Define galaxy
###########################################################################
# Check validity
assert gal in df_binned["ID"].values, f"{gal} not found in SAMI sample!"

# Load the DataFrame
df_gal = df_binned[df_binned["ID"] == gal]
df_gal.loc[df_gal["Number of components"] == 0, "Number of components"] = np.nan

###########################################################################
# Create the figure
###########################################################################
fig_collage = plt.figure(figsize=(18, 7))
ax_sdss = fig_collage.add_axes([l, b, w, h])
ax_im = fig_collage.add_axes([l, b + h + dh, w, h])
bbox = ax_im.get_position()
cax_im = fig_collage.add_axes([bbox.x0 + bbox.width * 0.035, bbox.y0 + bbox.height, bbox.width * 0.93, 0.025])
axs_bpt = []
axs_bpt.append(fig_collage.add_axes([l + w + dw, b + h + dh, w, h]))
axs_bpt.append(fig_collage.add_axes([l + w + dw + w, b + h + dh, w, h]))
axs_bpt.append(fig_collage.add_axes([l + w + dw + 2 * w, b + h + dh, w, h]))
axs_whav = []
axs_whav.append(fig_collage.add_axes([l + w + dw, b, w, h]))
axs_whav.append(fig_collage.add_axes([l + w + dw + w, b, w, h]))
axs_whav.append(fig_collage.add_axes([l + w + dw + 2 * w, b, w, h]))

###########################################################################
# Plot SDSS image and component map
###########################################################################
col_z = "Number of components"

# SDSS image
res = plot_sdss_image(df_gal, ax=ax_sdss)
if res is None:
    ax_sdss.text(s="Galaxy not in SDSS footprint",
                 x=0.5, y=0.5, horizontalalignment="center",
                 transform=ax_sdss.transAxes)

# Plot the number of components fitted.
plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
          PA_deg=0,
          col_z=col_z, 
          ax=ax_im, cax=cax_im, cax_orientation="horizontal", show_title=False)

# Text string showing basic info
sfr = df_snr.loc[gal, 'SFR (component 0)']
mstar = df_gal["log M_*"].unique()[0]
if np.isnan(sfr):
    sfr = "n/a"
else:
    sfr = f"{sfr:.3f}" + r" $\rm M_\odot\,yr^{-1}$"
    
if np.isnan(mstar):
    mstar = r"$\log \rm \, M_\odot = $ n/a"
else:
    mstar = r"$\log \rm \, M_\odot = $" + f"{mstar:.2f}"

t = axs_bpt[0].text(s=f"{gal}, {df_snr.loc[gal, 'Morphology']}, SFR = {sfr}, {mstar}, SNR = {df_snr.loc[gal, 'Median SNR (R, 2R_e)']:.2f}", 
    x=0.0, y=1.02, transform=axs_bpt[0].transAxes)

# Plot BPT diagram
col_y = "log O3"
for cc, col_x in enumerate(["log N2", "log S2", "log O1"]):
    # Plot full SAMI sample
    plot2dhistcontours(df=df_binned, 
                       col_x=f"{col_x} (total)",
                       col_y=f"{col_y} (total)", col_z="count", log_z=True,
                       alpha=0.5, cmap="gray_r",
                       ax=axs_bpt[cc], plot_colorbar=False)

    # Add BPT functions
    plot_BPT_lines(ax=axs_bpt[cc], col_x=col_x)    

    # Plot measurements for this galaxy
    plot2dscatter(df=df_gal,
                  col_x=f"{col_x} (total)",
                  col_y=f"{col_y} (total)",
                  col_z=None if col_z == "Number of components" else col_z,
                  marker=markers[0], ax=axs_bpt[cc], 
                  cax=None,
                  markersize=20, 
                  markerfacecolor=component_colours[0] if col_z == "Number of components" else None, 
                  markeredgecolor="black",
                  plot_colorbar=False)

# Decorations
[ax.grid() for ax in axs_bpt]
[ax.set_ylabel("") for ax in axs_bpt[1:]]
[ax.set_yticklabels([]) for ax in axs_bpt[1:]]
[ax.set_xticks(ax.get_xticks()[:-1]) for ax in axs_bpt[:-1]]
for ax in axs_bpt:
    _ = [c.set_rasterized(True) for c in ax.collections]
_ = [c.set_rasterized(True) for c in ax_im.collections]

###########################################################################
# Plot WHAN, WHAV and WHAV* diagrams.
###########################################################################
# Plot LZIFU measurements
for cc, col_x in enumerate(["log N2", "sigma_gas - sigma_*", "v_gas - v_*"]):
    # Plot full SAMI sample
    plot2dhistcontours(df=df_binned, 
                       col_x=f"{col_x} (total)" if col_x == "log N2" else f"{col_x}",
                       col_y=f"log HALPHA EW (total)" if col_x == "log N2" else f"log HALPHA EW",
                       col_z="count", log_z=True,
                       alpha=0.5, cmap="gray_r", ax=axs_whav[cc],
                       plot_colorbar=False)

# WHAN diagram
plot2dscatter(df=df_gal,
              col_x=f"log N2 (total)",
              col_y=f"log HALPHA EW (total)",
              col_z=None if col_z == "Number of components" else col_z,
              marker=markers[0], ax=axs_whav[0], 
              cax=None,
              markersize=20, 
              markerfacecolor=component_colours[0] if col_z == "Number of components" else None, 
              markeredgecolor="black",
              plot_colorbar=False)

# Kinematics 
for cc, col_x in enumerate(["sigma_gas - sigma_*", "v_gas - v_*"]):
    # Plot the data for this galaxy
    for ii in range(3):
        plot2dscatter(df=df_gal,
                      col_x=f"{col_x} (component {ii})",
                      col_y=f"log HALPHA EW (component {ii})",
                      col_z=None if col_z == "Number of components" else col_z,
                      marker=markers[ii], ax=axs_whav[cc + 1], 
                      cax=None,
                      markersize=20, 
                      markerfacecolor=component_colours[ii] if col_z == "Number of components" else None, 
                      markeredgecolor="black",
                      plot_colorbar=False)

# Decorations
[ax.grid() for ax in axs_whav]
[ax.set_ylabel("") for ax in axs_whav[1:]]
[ax.set_yticklabels([]) for ax in axs_whav[1:]]
[ax.set_xticks(ax.get_xticks()[:-1]) for ax in axs_whav[:-1]]
[ax.axvline(0, ls="--", color="k") for ax in axs_whav[1:]]
for ax in axs_whav:
    _ = [c.set_rasterized(True) for c in ax.collections]

# Legend
legend_elements = [Line2D([0], [0], marker=markers[ii], 
                          color="none", markeredgecolor="black",
                          label=f"Component {ii}",
                          markerfacecolor=component_colours[ii], markersize=5) for ii in range(3)]
axs_bpt[-1].legend(handles=legend_elements, fontsize="x-small", loc="upper right")


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gal["x, y (pixels)"] = list(zip(df_gal["x (projected, arcsec)"] / as_per_px, df_gal["y (projected, arcsec)"] / as_per_px))
